In [19]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Load the wiki page and parse out the table with ```wikitable``` CSS class

In [20]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')
table = soup.find_all("table", class_="wikitable")[0]

### Parse the table rows and columns and convert them into an array

In [21]:
records = []
for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) == 3:
            records.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
data_array = np.asarray(records)

### Build the data frame using the array with the specified column titles

In [22]:
df = pd.DataFrame(data_array, columns = ['PostalCode', 'Borough', 'Neighborhood'])

### Drop the items with no ```Borough``` name

In [23]:
df = df[df.Borough != 'Not assigned']

### Replace _Not assigned_ values in  ```Neighborhood``` column with the value from  ```Borough```  column 

In [24]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

### Finally group the items by  ```PostalCode``` value and join  ```Neighborhood``` values

In [25]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

### Print the shape of data frame

In [26]:
df.shape

(103, 3)

### Read the latitude and longitude information for postal codes into a data frame

In [27]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')

### Join the columns from both data frames on  ```PostalCode```  so that each postal code in our data frme will have a longitude and lattitude columns

In [28]:
df = df.join(df_geo.set_index('Postal Code'), on='PostalCode')

### Check the shape of the new data frame to make sure we have 2 additional columns.

In [29]:
df.shape

(103, 5)